In [ ]:
# !pip install groq pandas

In [ ]:
import csv
from groq import Groq

# Initialize the Groq client
client = Groq(
    api_key="API_KEY"  # Replace with your actual API key
)

# Function to call Groq API with LLAMA 3 model
def call_groq_api(prompt, model="llama3-70b-8192"):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model=model,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error calling Groq API: {e}")
        return None

# Function to read notes from a text file
def read_notes(input_file):
    with open(input_file, 'r') as file:
        return file.read()

# Function to craft the prompt for generating quiz from notes
def generate_quiz_from_notes(notes):
    prompt = f"""Generate a quiz from the following notes. The quiz should contain exactly 15 questions, 
    with each question having 3 options (A, B, C, D) and specify the correct answer. Format each question as follows:

    **Question X**
    [Question text] (Category)
    A) [Option A]
    B) [Option B]
    C) [Option C]
    D) [Option D]
    Correct Answer: [Correct option]

    Where X is the question number. Ensure there's a blank line between each question.
    Also mark the question with its category like [Question text] (Category)
    Categories:
    - Theoretical : 3 Questions
    - Understanding : 3 Questions
    - Pseudocode : 3 Questions
    - Remembering : 3 Questions
    - Logical Reasoning : 3 Questions
    Each category should have minimum 3 questions

    Notes:\n{notes}"""
    
    return call_groq_api(prompt)

# Function to process the API response and save the quiz to a CSV
def save_quiz_to_csv(quiz_text, output_file):
    print("Raw Quiz Text from API:\n", quiz_text)
    questions = []
    current_question = {}
    
    lines = quiz_text.split("\n")
    for line in lines:
        line = line.strip()
        if line.startswith("**Question"):
            if current_question:
                questions.append(current_question)
            current_question = {"Question": "", "Option A": "", "Option B": "", "Option C": "","Option D": "", "Correct Answer": ""}
        elif line.startswith("What") or line.startswith("Which") or line.endswith("?"):
            current_question["Question"] = line
        elif line.startswith("A)"):
            current_question["Option A"] = line[3:].strip()
        elif line.startswith("B)"):
            current_question["Option B"] = line[3:].strip()
        elif line.startswith("C)"):
            current_question["Option C"] = line[3:].strip()
        elif line.startswith("D)"):
            current_question["Option D"] = line[3:].strip()
        elif line.startswith("Correct Answer:"):
            current_question["Correct Answer"] = line[16:].strip()
    
    if current_question:
        questions.append(current_question)
    
    # Write the questions to a CSV file
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Question", "Option A", "Option B", "Option C","Option D", "Correct Answer"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for question in questions:
            writer.writerow(question)
    
    print(f"Quiz saved to {output_file} with {len(questions)} questions.")

# Main function to tie everything together
def create_quiz(input_file, output_file):
    notes = read_notes(input_file)
    quiz_text = generate_quiz_from_notes(notes)
    
    if quiz_text:
        save_quiz_to_csv(quiz_text, output_file)
    else:
        print("Failed to generate quiz.")

# Example usage
input_file = "Module1_Notes_to_Quiz/Notes.txt"  # Your text file with notes
output_file = "Module1_Notes_to_Quiz/quiz.csv"  # The output CSV file
create_quiz(input_file, output_file)


Raw Quiz Text from API:
 Here are the 15 questions based on the provided notes:

**Question 1**
Time complexity of searching in an unsorted array (Theoretical)
A) O(log n)
B) O(n)
C) O(n log n)
D) O(1)
Correct Answer: B) O(n)

**Question 2**
What is the main disadvantage of arrays? (Understanding)
A) They are too flexible
B) They have a fixed size
C) They are too slow
D) They are too complex
Correct Answer: B) They have a fixed size

**Question 3**
What is the time complexity of insertion or deletion in an array? (Theoretical)
A) O(1)
B) O(log n)
C) O(n)
D) O(n log n)
Correct Answer: C) O(n)

**Question 4**
What is the primary advantage of doubly linked lists over singly linked lists? (Understanding)
A) Faster search time
B) Easier insertion and deletion
C) Bidirectional traversal
D) Less memory usage
Correct Answer: C) Bidirectional traversal

**Question 5**
What is the purpose of the additional pointer in a doubly linked list node? (Pseudocode)
A) To point to the next node
B) To poin